In [9]:
#definition of the yaw cases

yaw_cases = {
    "0deg_baseline_csv": 0.0,
    "15deg_baseline_csv": 15.0,
    "30deg_baseline_csv": 30.0,
    "45deg_baseline_csv": 45.0,
}


In [10]:
import os 
import numpy as np 
import pandas as pd

def load_snapshot(folder_path):
    """Load a single timestep folder: U, p, k, nut, omega."""

    U = pd.read_csv(os.path.join(folder_path, "U.csv"), header=None).values  # Nx3
    p = pd.read_csv(os.path.join(folder_path, "p.csv"), header=None).values  # Nx1
    k = pd.read_csv(os.path.join(folder_path, "k.csv"), header=None).values  # Nx1
    nut = pd.read_csv(os.path.join(folder_path, "nut.csv"), header=None).values  # Nx1
    omega = pd.read_csv(os.path.join(folder_path, "omega.csv"), header=None).values  # Nx1

    # Concatenate all fields
    fields = np.hstack([U, p, k, nut, omega])  # shape: (N, 3+1+1+1+1)
    flat = fields.flatten()  # shape: (N * 7,)
    return flat


In [11]:

def load_yaw_case(base_dir, yaw_angle):
    """Load all snapshots for a specific yaw case."""
    X_list = []
    yaw_list = []

    time_dirs = sorted([d for d in os.listdir(base_dir) if d.isdigit()])

    for t in time_dirs:
        snapshot_dir = os.path.join(base_dir, t)
        flat_vec = load_snapshot(snapshot_dir)

        X_list.append(flat_vec)
        yaw_list.append(yaw_angle)

    return np.array(X_list), np.array(yaw_list).reshape(-1, 1)


In [13]:
yaw_folders = {
    "0deg_baseline_csv": 0.0,
    "15deg_baseline_csv": 15.0,
    "30deg_baseline_csv": 30.0,
    "45deg_baseline_csv": 45.0,
}

X_all = []
yaw_all = []

base_path = "../"

for folder, yaw_angle in yaw_folders.items():
    full_path = os.path.join(base_path, folder)
    Xi, yi = load_yaw_case(full_path, yaw_angle)

    X_all.append(Xi)
    yaw_all.append(yi)

X_all = np.vstack(X_all)
yaw_all = np.vstack(yaw_all)

print("Dataset shape:", X_all.shape)
print("Yaw labels shape:", yaw_all.shape)


/tmp/ipykernel_47441/1406239696.py:8: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  U = pd.read_csv(os.path.join(folder_path, "U.csv"), header=None).values  # Nx3
/tmp/ipykernel_47441/1406239696.py:8: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  U = pd.read_csv(os.path.join(folder_path, "U.csv"), header=None).values  # Nx3
/tmp/ipykernel_47441/1406239696.py:8: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  U = pd.read_csv(os.path.join(folder_path, "U.csv"), header=None).values  # Nx3
/tmp/ipykernel_47441/1406239696.py:8: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtype option on import or set low_memory=False.
  U = pd.read_csv(os.path.join(folder_path, "U.csv"), header=None).values  # Nx3
/tmp/ipykernel_47441/1406239696.py:8: DtypeWarning: Columns (0,1,2,3) have mixed types. Specify dtyp

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (21,) + inhomogeneous part.

In [ ]:
from sklearn.decomposition import PCA

latent_dim = 50  # try 20–100

pca = PCA(n_components=latent_dim)
Z = pca.fit_transform(X_all)

print("Latent Z shape:", Z.shape)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

class YawToLatent(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, latent_dim)
        )

    def forward(self, yaw):
        return self.net(yaw)


model = YawToLatent(latent_dim)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MSELoss()

dataset = TensorDataset(torch.tensor(yaw_all, dtype=torch.float32),
                        torch.tensor(Z, dtype=torch.float32))

loader = DataLoader(dataset, batch_size=8, shuffle=True)

for epoch in range(500):
    for yaw_batch, z_batch in loader:
        optimizer.zero_grad()
        z_pred = model(yaw_batch)
        loss = loss_fn(z_pred, z_batch)
        loss.backward()
        optimizer.step()

    if epoch % 50 == 0:
        print("Epoch", epoch, "loss", loss.item())


In [ ]:
yaw_query = torch.tensor([[22.5]], dtype=torch.float32)

Z_pred = model(yaw_query).detach().numpy()
X_pred = pca.inverse_transform(Z_pred)[0]


In [ ]:
num_features = 3 + 1 + 1 + 1 + 1  # U + p + k + nut + omega
N_cells = X_all.shape[1] // num_features

pred_fields = X_pred.reshape(N_cells, num_features)

Ux = pred_fields[:, 0]
Uy = pred_fields[:, 1]
Uz = pred_fields[:, 2]
p  = pred_fields[:, 3]
k  = pred_fields[:, 4]
nut= pred_fields[:, 5]
omega=pred_fields[:, 6]
